# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 24 2023 8:38AM,258105,12,HH22324,Hush Hush,Released
1,Feb 24 2023 8:38AM,258104,18,EMERSA-9199787,"Emersa Waterbox, LLC",Released
2,Feb 24 2023 8:30AM,258091,10,0086428397,ISDIN Corporation,Released
3,Feb 24 2023 8:30AM,258090,10,0086428205,ISDIN Corporation,Released
4,Feb 24 2023 8:30AM,258090,10,0086428206,ISDIN Corporation,Released
5,Feb 24 2023 8:30AM,258090,10,0086428207,ISDIN Corporation,Released
6,Feb 24 2023 8:30AM,258091,10,0086427907,ISDIN Corporation,Released
7,Feb 24 2023 8:30AM,258091,10,0086427906,ISDIN Corporation,Released
8,Feb 24 2023 8:30AM,258091,10,0086428401,ISDIN Corporation,Released
9,Feb 24 2023 8:30AM,258090,10,0086427466,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,258101,Released,1
33,258102,Released,1
34,258103,Released,3
35,258104,Released,1
36,258105,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258101,NaN,NaN,1.0
258102,NaN,NaN,1.0
258103,NaN,NaN,3.0
258104,NaN,NaN,1.0
258105,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258014,14.0,1.0,2.0
258031,4.0,2.0,1.0
258039,5.0,13.0,2.0
258041,0.0,0.0,1.0
258044,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258014,14,1,2
258031,4,2,1
258039,5,13,2
258041,0,0,1
258044,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258014,14,1,2
1,258031,4,2,1
2,258039,5,13,2
3,258041,0,0,1
4,258044,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258014,14,1,2
1,258031,4,2,1
2,258039,5,13,2
3,258041,,,1
4,258044,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 24 2023 8:38AM,258105,12,Hush Hush
1,Feb 24 2023 8:38AM,258104,18,"Emersa Waterbox, LLC"
2,Feb 24 2023 8:30AM,258091,10,ISDIN Corporation
3,Feb 24 2023 8:30AM,258090,10,ISDIN Corporation
57,Feb 24 2023 8:30AM,258089,10,"Methapharm, Inc."
67,Feb 24 2023 8:28AM,258103,19,"Emersa Waterbox, LLC"
70,Feb 24 2023 8:27AM,258102,16,TASA USA Inc.
71,Feb 24 2023 8:21AM,258101,16,Sartorius Bioprocess Solutions
72,Feb 24 2023 8:16AM,258100,16,Sartorius Bioprocess Solutions
73,Feb 24 2023 8:15AM,258099,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 24 2023 8:38AM,258105,12,Hush Hush,,,1
1,Feb 24 2023 8:38AM,258104,18,"Emersa Waterbox, LLC",,,1
2,Feb 24 2023 8:30AM,258091,10,ISDIN Corporation,,,4
3,Feb 24 2023 8:30AM,258090,10,ISDIN Corporation,,,56
4,Feb 24 2023 8:30AM,258089,10,"Methapharm, Inc.",,,5
5,Feb 24 2023 8:28AM,258103,19,"Emersa Waterbox, LLC",,,3
6,Feb 24 2023 8:27AM,258102,16,TASA USA Inc.,,,1
7,Feb 24 2023 8:21AM,258101,16,Sartorius Bioprocess Solutions,,,1
8,Feb 24 2023 8:16AM,258100,16,Sartorius Bioprocess Solutions,,,1
9,Feb 24 2023 8:15AM,258099,19,"AdvaGen Pharma, Ltd",,,23


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 24 2023 8:38AM,258105,12,Hush Hush,1,,
1,Feb 24 2023 8:38AM,258104,18,"Emersa Waterbox, LLC",1,,
2,Feb 24 2023 8:30AM,258091,10,ISDIN Corporation,4,,
3,Feb 24 2023 8:30AM,258090,10,ISDIN Corporation,56,,
4,Feb 24 2023 8:30AM,258089,10,"Methapharm, Inc.",5,,
5,Feb 24 2023 8:28AM,258103,19,"Emersa Waterbox, LLC",3,,
6,Feb 24 2023 8:27AM,258102,16,TASA USA Inc.,1,,
7,Feb 24 2023 8:21AM,258101,16,Sartorius Bioprocess Solutions,1,,
8,Feb 24 2023 8:16AM,258100,16,Sartorius Bioprocess Solutions,1,,
9,Feb 24 2023 8:15AM,258099,19,"AdvaGen Pharma, Ltd",23,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 24 2023 8:38AM,258105,12,Hush Hush,1,,
1,Feb 24 2023 8:38AM,258104,18,"Emersa Waterbox, LLC",1,,
2,Feb 24 2023 8:30AM,258091,10,ISDIN Corporation,4,,
3,Feb 24 2023 8:30AM,258090,10,ISDIN Corporation,56,,
4,Feb 24 2023 8:30AM,258089,10,"Methapharm, Inc.",5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 24 2023 8:38AM,258105,12,Hush Hush,1.0,NaN,NaN
1,Feb 24 2023 8:38AM,258104,18,"Emersa Waterbox, LLC",1.0,NaN,NaN
2,Feb 24 2023 8:30AM,258091,10,ISDIN Corporation,4.0,NaN,NaN
3,Feb 24 2023 8:30AM,258090,10,ISDIN Corporation,56.0,NaN,NaN
4,Feb 24 2023 8:30AM,258089,10,"Methapharm, Inc.",5.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 24 2023 8:38AM,258105,12,Hush Hush,1.0,0.0,0.0
1,Feb 24 2023 8:38AM,258104,18,"Emersa Waterbox, LLC",1.0,0.0,0.0
2,Feb 24 2023 8:30AM,258091,10,ISDIN Corporation,4.0,0.0,0.0
3,Feb 24 2023 8:30AM,258090,10,ISDIN Corporation,56.0,0.0,0.0
4,Feb 24 2023 8:30AM,258089,10,"Methapharm, Inc.",5.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4387067,172.0,16.0,23.0
12,516149,2.0,0.0,0.0
16,774303,3.0,0.0,0.0
18,258104,1.0,0.0,0.0
19,1548534,61.0,0.0,0.0
20,258052,1.0,0.0,0.0
22,258097,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4387067,172.0,16.0,23.0
1,12,516149,2.0,0.0,0.0
2,16,774303,3.0,0.0,0.0
3,18,258104,1.0,0.0,0.0
4,19,1548534,61.0,0.0,0.0
5,20,258052,1.0,0.0,0.0
6,22,258097,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,172.0,16.0,23.0
1,12,2.0,0.0,0.0
2,16,3.0,0.0,0.0
3,18,1.0,0.0,0.0
4,19,61.0,0.0,0.0
5,20,1.0,0.0,0.0
6,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,172.0
1,12,Released,2.0
2,16,Released,3.0
3,18,Released,1.0
4,19,Released,61.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20,22
Status,,,,,,,
Completed,23.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,16.0,0.0,0.0,0.0,0.0,0.0,1.0
Released,172.0,2.0,3.0,1.0,61.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20,22
0,Completed,23.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,16.0,0.0,0.0,0.0,0.0,0.0,1.0
2,Released,172.0,2.0,3.0,1.0,61.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20,22
0,Completed,23.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,16.0,0.0,0.0,0.0,0.0,0.0,1.0
2,Released,172.0,2.0,3.0,1.0,61.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()